In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
np.random.seed(151)

# 宿題

# データのロード

In [ ]:
train = pd.read_csv("data/train_enshu1.csv")
test = pd.read_csv("data/test_enshu1.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train = pd.pivot_table(train, index="Date/Time", columns="Base", values="count")

In [ ]:
test = pd.pivot_table(test, index="Date/Time", columns="Base", values="count")

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test.shape

## データ正規化

In [ ]:
from sklearn.preprocessing import MinMaxScaler

1週間の移動平均からの乖離を計算します。

In [ ]:
diff_ma_train = train - train.rolling(7).mean()
diff_ma_test = test - test.rolling(7).mean()

初めの6日分がNaNになるので、dropします。

In [ ]:
diff_ma_train.dropna(inplace=True)
diff_ma_test.dropna(inplace=True)

-1から1の範囲にscaleします。

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))

In [ ]:
train_f = scaler.fit_transform(diff_ma_train.values.astype(np.float32))

In [ ]:
test_f = scaler.transform(diff_ma_test.values.astype(np.float32))

In [ ]:
train_f.shape

In [ ]:
test_f.shape

## データをGRUが読み込める形式に変換

学習データとテストデータを作成します。

In [ ]:
train_X = train_f[:-1].T.reshape((5, 146, 1))

In [ ]:
train_Y = train_f[1:].T.reshape((5, 146, 1))

In [ ]:
test_X = test_f[:-1].T.reshape((5, 23, 1))

In [ ]:
test_Y = test_f[1:].T.reshape((5, 23, 1))

In [ ]:
train_X.shape

## モデルの作成

In [ ]:
import keras

In [ ]:
from keras.models import Sequential
from keras.layers import GRU, TimeDistributed, Dense, Dropout

GRUを使ってモデルを作りましょう。  
GRUについては以下のマニュアルを参照してください。  
https://keras.io/ja/layers/recurrent/#gru

In [ ]:
model = Sequential()
# TODO 演習1のモデルから、LSTMレイヤーの部分をGRUレイヤーに変更したモデルを作成してください。
model.compile(loss="mean_squared_error", optimizer="adam")

## 学習

LSTMが最高性能を出した反復回数(1137回)と同じ回数だけ反復してみましょう。

In [ ]:
%%time
history = model.fit(train_X, train_Y, nb_epoch=1137, validation_split=0.1)

In [ ]:
pd.DataFrame(history.history).plot()

## 評価

### 学習データに対するFitを確認

In [ ]:
pred_Y_train = model.predict(train_Y)

In [ ]:
pred_Y_train.shape

In [ ]:
train_Y.shape

In [ ]:
pred_train = pred_Y_train.reshape((5, 146))
ref_train = train_Y.reshape((5, 146))

In [ ]:
for i in range(5):
    pd.DataFrame(dict(pred=pred_train[i], ref=ref_train[i])).plot()

### テストデータに対するFitを確認

In [ ]:
pred_Y = model.predict(test_X)

In [ ]:
pred_Y.shape

In [ ]:
test_Y.shape

In [ ]:
pred = pred_Y.reshape((5, 23))
ref = test_Y.reshape((5, 23))

In [ ]:
for i in range(5):
    pd.DataFrame(dict(pred=pred[i], ref=ref[i])).plot()

### モデル出力から予測値に変換してみましょう

In [ ]:
test.rolling(7).mean().iloc[6:].shape, scaler.inverse_transform(pred.T).shape

In [ ]:
predicted_final = test.rolling(7).mean().iloc[7:] + scaler.inverse_transform(pred.T)

In [ ]:
predicted_final.shape, test.values.shape

In [ ]:
for i in range(5):
    pd.DataFrame(dict(pred=predicted_final.values[:, i], ref=test.values[7:, i])).plot()